In [1]:
%load_ext autoreload
%autoreload 2

In [4]:
import os
os.environ['CUDA_DEVICE_ORDER'] = 'PCI_BUS_ID'
os.environ['CUDA_VISIBLE_DEVICES'] = '2'
import numpy as np
import tensorflow as tf
import functools
import sys
home = os.path.expanduser("~")
user = home.split('/')[-1]
sys.path.append(home + '/alaska_github/src/')
from tools.tf2onnx_utils import *
from tools.models import *
from tools.jpeg_utils import *
from tools.train_estimator import *
from tools.tf_utils import *
import pickle
from tqdm import tqdm

In [5]:
QF = 95

checkpoint_prefix = home + '/alaska_github/models/feature_extractors/QF'+str(QF)+'/tensorflow/'
checkpoint_path = checkpoint_prefix + 'model.cpkt-color_separated'

data_prefix = home + '/alaska_github/data/QF'+str(QF)+'/tiles/JPEG/'
data_dict = {'COVER':data_prefix+'COVER/',
            'JUNI':data_prefix+'JUNI/',
            'UED':data_prefix+'UED/',
            'NSF5':data_prefix+'NSF5/',
            'EBS':data_prefix+'EBS/'}

with open('IL_val.p', 'rb') as f:
    IL_val = pickle.load(f)
with open('IL_train.p', 'rb') as f:
    IL_train = pickle.load(f)

In [ ]:
branch = 'YCrCb'
LOG_DIR =  home + '/alaska_github/src/LogFiles/QF'+str(QF)+'/'+branch+'/'
boundaries = [20000, 100000]
values = [0.0001, 0.001, 0.0001]
max_iter = 200000
save_interval = 1000
stego_schemes = ['EBS', 'JUNI', 'NSF5', 'UED']
priors = [0.15, 0.4, 0.15, 0.3]
valid_batch_size = 100
train_batch_size = 64
num_runner_threads = 10
hidden_sizes = None

warm_start_checkpoint = checkpoint_path
load_checkpoint = None

train_estimator(SR_net_multiclass, hidden_sizes, 5, LOG_DIR, boundaries, values, save_interval,
                    warm_start_checkpoint, load_checkpoint, branch, 
                    data_dict['COVER'], data_dict, stego_schemes, priors,
                    IL_train, IL_val, valid_batch_size, 
                    train_batch_size, num_runner_threads,
                    max_iter, gen_train, gen_valid)